In [1]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
from time import time

twenty_train = fetch_20newsgroups(subset='train', shuffle=True, remove = ('headers', 'footers'))
targets_train = twenty_train['target']

twenty_test = fetch_20newsgroups(subset = 'test', remove = ('headers', 'footers'))
targets_test = twenty_test['target']

groups = np.arange(20)
priori = np.mean(targets_train == groups.reshape(-1, 1), axis = 1)

#base = np.arange(targets_test.size)
#val_idx = np.random.choice(base, size = int(targets_test.size / 2), replace = False)
#test_idx = base[np.all(base != val_idx.reshape(-1, 1), axis = 0)]
#val_idx = [0]#np.arange(int(targets_test.size / 2))
test_idx = np.arange(targets_test.size)

def attempt(smoothing, max_weight, **kwargs):
    cv = CountVectorizer(**kwargs, stop_words = 'english')
    fitted = cv.fit_transform(twenty_train['data'])
    
    #f2 = fitted.toarray()
    #words = np.array(cv.get_feature_names())

    probabilities = np.zeros((20, fitted.shape[1]))
    
    #df = np.count_nonzero(f2, axis = 0) / targets_train.size
    #plt.hist(df * 100, bins = 80)
    #plt.yscale('log')
    #plt.xlabel('%')
    #plt.ylabel('Repeticiones')
   
    for group in groups:
        row = fitted[targets_train == group].sum(axis = 0) + smoothing
        row[0, np.array(row == row.max())[0]] *= max_weight
        probabilities[group] = row[0] / row.sum()
        
    #np.delete(f2)
    #del words
    fitted_test = cv.transform(twenty_test['data'])#.toarray().T
        
    probs_test = fitted_test.dot(np.log(probabilities).T) + np.log(priori)

    pred_test = np.argmax(probs_test, axis = 1)

    acc_test = np.mean(pred_test == targets_test)
    
    return {'Test': (acc_test * 100).round(2)}

In [ ]:
d = np.arange(.01, .15, .005)
s = np.arange(1e-3, .11, .01)

for df in d:
    for smooth in s:
        print(df, smooth)
        print(attempt(min_df = 1, max_df = df, smoothing = smooth, max_weight = 2.5))

0.01 0.001
{'Test': 72.73}
0.01 0.011
{'Test': 73.35}
0.01 0.020999999999999998
{'Test': 73.57}
0.01 0.030999999999999996
{'Test': 73.83}
0.01 0.040999999999999995
{'Test': 73.79}
0.01 0.05099999999999999
{'Test': 73.9}
0.01 0.06099999999999999
{'Test': 73.94}
0.01 0.071
{'Test': 73.98}
0.01 0.08099999999999999
{'Test': 74.03}
0.01 0.09099999999999998
{'Test': 73.99}
0.01 0.10099999999999998
{'Test': 73.99}
0.015 0.001
{'Test': 73.88}
0.015 0.011
{'Test': 74.71}
0.015 0.020999999999999998
{'Test': 74.92}
0.015 0.030999999999999996
{'Test': 75.08}
0.015 0.040999999999999995
{'Test': 75.15}
0.015 0.05099999999999999
{'Test': 75.23}
0.015 0.06099999999999999
{'Test': 75.24}
0.015 0.071
{'Test': 75.35}
0.015 0.08099999999999999
{'Test': 75.28}
0.015 0.09099999999999998
{'Test': 75.35}
0.015 0.10099999999999998
{'Test': 75.37}
0.019999999999999997 0.001
{'Test': 74.31}
0.019999999999999997 0.011
{'Test': 75.04}
0.019999999999999997 0.020999999999999998
{'Test': 75.31}
0.019999999999999997 0

In [ ]:
sens_tot = []
especif_tot = []
vp_tot = []
vn_tot = []

for i in range(20):
    targets_ok = targets_test == i
    pred_ok = pred == i
    VP = np.sum(targets_ok & pred_ok)
    VN = np.sum(~targets_ok & ~pred_ok)
    FP = np.sum(~targets_ok & pred_ok)
    FN = np.sum(targets_ok & ~pred_ok)
    
    sens_tot.append(VP / (VP + FN))
    especif_tot.append(VN / (VN + FP))
    vp_tot.append(VP / (VP + FP))
    vn_tot.append(VN / (VN + FN))
    
print(f'Sensibilidad: {np.round(np.mean(sens_tot) * 100, 2)}%')

# No tiene sentido, porque son 20 clases. Obvio que va a dar alto.
print(f'Especificidad: {np.round(np.mean(especif_tot) * 100, 2)}%') 

print(f'Valor predictivo positivo: {np.round(np.mean(vp_tot) * 100, 2)}%')

# No tiene sentido, porque son 20 clases. Obvio que va a dar alto.
print(f'Valor predictivo negativo: {np.round(np.mean(vn_tot) * 100, 2)}%')

## Preguntas
- $\textbf{Parte 1}$:
    - ¿Va bien 79.3%?
    - ¿Qué más podemos hacer para subirlo?
    - ¿Está bien usar accuracy?
    - ¿Para las otras métricas está bien sacar el promedio?
    
- $\textbf{Parte 2}$:
    - ¿Podemos sacar el priori si da peores resultados?
    - ¿Está bien elegida la sensibilidad como métrica?
    - ¿Está bien reemplazar a los NaN por la media según la clase?    

In [ ]:
print(twenty_train['DESCR'])

In [ ]:
print(twenty_train['target_names'])